<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment02/Assignment2_PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALLING REQUIRED DEPENDENCIES

In [ ]:
#importing Google drive to load the dataset
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#installing wandb
!pip install wandb

In [ ]:
#unzipping the dataset which is stored in the Drive
!unzip '/content/drive/MyDrive/nature_12K'

# IMPORTING LIBRARIES

In [4]:
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import InceptionV3, InceptionResNetV2, ResNet50, Xception

# SPLITTING THE TRAINING AND VALIDATION SETS
As in the previous part, we use image_dataset_from_directory module to split the training and the validation datasets. Here, 90% of the training dataset is used for training and 10% is used for validation.

In [5]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/inaturalist_12K/train', 
                                                                    validation_split=0.1, seed = 123, subset = "training")

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory('/content/inaturalist_12K/train', 
                                                                    validation_split=0.1, seed = 123, subset = "validation")

Found 9999 files belonging to 10 classes.
Using 9000 files for training.
Found 9999 files belonging to 10 classes.
Using 999 files for validation.


# DATA AUGMENTATION
Four augmentations, viz. Random Flip, Random Crop, Random Rotation, Random Translation are being used here. These are chosen because these are the possible modifications which the model will see during the test time and thus, it is important that the model recognises these modified images correctly as well.

In [6]:
IMG_SIZE = 256

data_aug = tf.keras.Sequential([preprocessing.RandomFlip(), #Random Flip
                                preprocessing.RandomCrop(IMG_SIZE, IMG_SIZE), #Random Crop
                                preprocessing.RandomRotation(factor = (-0.2, 0.2)), #Random Rotation
                                preprocessing.RandomTranslation(height_factor=(-0.2, 0.2), width_factor=(-0.2,0.2)), #Random Translation
                              ])

# DEFINING THE MODEL
First, I am running a dummy example as a template for the final code. As an illustration, I'm using InceptionV3 as the base model. Apart from the base model, I have added an Average Pooling Layer and a softmax output layer.

In [7]:
preprocess_input = tf.keras.applications.inception_v3.preprocess_input #the preprocessing input function for InceptionV3
base_model = InceptionV3(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)) #defining the base model
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()  #average pooling layer
prediction_layer = tf.keras.layers.Dense(10, activation="softmax")  #final softmax layer

87916544/87910968 [==============================] - 1s 0us/step


In [8]:
#Model Definition
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_aug(inputs)
x = preprocess_input(x)
x = base_model(x)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)

# EXPERIMENT-1
Base Model: InceptionV3 <br>
Learning rate = 1e-3 <br>
Optimizer: Adam <br>
Freezing 275/311 layers <br>
Number of epochs: 10

In [9]:
#Defining the hyperparameters such as optimizer and learning rate
model = tf.keras.Model(inputs, outputs)
learning_rate = 1e-3
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics='accuracy')

In [ ]:
#Number of layers in the base model so that we can freeze a certain number of layers appropriately
base_model.trainable = True
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
#Freezing 275/311 layers
fine_tune_at = 275
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
epochs = 10

model.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

# EXPERIMENT-2
Base Model: InceptionV3 <br>
Learning rate = 1e-3 <br>
Optimizer: Adam <br>
Freezing 300/311 layers <br>
Number of epochs: 10

In [ ]:
#defining a function so that it would be easier to run more experiments
def pre_train_adam(fine_tune_at):
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

  model = tf.keras.Model(inputs, outputs)
  learning_rate = 1e-3
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics='accuracy')
  epochs = 10
  model.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

In [ ]:
pre_train_adam(300)

# EXPERIMENT-3
Base Model: InceptionV3 <br>
Learning rate = 1e-3 <br>
Optimizer: NAG <br>
Freezing 300/311 layers <br>
Number of epochs: 10

In [ ]:
#defining a similar function
def pre_train_nag(fine_tune_at, learning_rate=1e-3, epochs=10):
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

  model = tf.keras.Model(inputs, outputs)

  model.compile(optimizer=tf.keras.optimizers.SGD(lr=learning_rate, decay=learning_rate/epochs, nesterov=True),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics='accuracy')
  
  model.fit(train_dataset, validation_data = validation_dataset, epochs = epochs)

In [ ]:
pre_train_nag(300)

# WANDB SWEEP FUNCTION

In [ ]:
def swp():
"""
  List of various hyperparameters and their values:
  base: base model, string, "IncV3" for InceptionV3, "IncResV2" for InceptionResNetV2, "Res50" for ResNet50 and "Xcep" for Xception
  fine_tune_at_fraction: fraction of total number of  layers to be frozen, chosen among 1,0.95,0.9,0.85,0.8
  epochs: Total number of epochs, chosen among 5,10,15
  data_augmentation: whether data augmentation has to be included or not, "Y" if it has to be included, "N" if it need not be included
  dropout: fraction of nodes for dropout, chosen among 0,0.2,0.3,0.4
  optimizer: optimizer to be used for training, "adam" for Adam optimizer, "nadam" for Nadam optimizer, "rmsprop" for RMSProp optimizer and "nag" for NAG optimizer.

"""
#---------DEFAULT HYPERPARAMETERS-----------------------------------------------
  hyperparameter_defaults = dict(
        base = "IncV3",
        fine_tune_at_fraction = 0.9,
        optimizer = 'nag',
        epochs = 10,
        data_augmentation = "Y",
        dropout = 0.2
      )

#----------------SETTING UP WANDB-----------------------------------------------
  wandb.init(project="Assignment 2", config=hyperparameter_defaults)
  config = wandb.config
  wandb.run.name = "{}_base_{}_finetune_{}_opt_{}_epoch_{}_DataAug_{}_dropout".format(config.base, config.fine_tune_at_fraction,config.optimizer, config.epochs, config.data_augmentation, config.dropout)

#----------------BASE MODEL DEFINITION AND INPUT PREPROCESSING------------------  
  if config.base == "IncV3": #InceptionV3
    preprocess_input = tf.keras.applications.inception_v3.preprocess_input
    base_model = InceptionV3(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

  elif config.base == "IncResV2": #InceptionResNetV2
    preprocess_input = tf.keras.applications.inception_resnet_v2.preprocess_input
    base_model = InceptionResNetV2(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
  
  elif config.base == "Res50": #ResNet50
    preprocess_input = tf.keras.applications.resnet.preprocess_input
    base_model = ResNet50(include_top=False, input_shape=(IMG_SIZE,IMG_SIZE,3))

  elif config.base == "Xcep": #Xception
    preprocess_input = tf.keras.applications.xception.preprocess_input
    base_model = Xception(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

#---------------DEFINING THE REST OF THE MODEL----------------------------------
  global_average_layer = tf.keras.layers.GlobalAveragePooling2D()  #Average pooling layer
  prediction_layer = tf.keras.layers.Dense(10, activation="softmax") #softmax output layer

  inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  if config.data_augmentation == "Y": #including data augmentation
    x = data_aug(inputs) 
    x = preprocess_input(x)
    x = base_model(x)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(config.dropout)(x)
    outputs = prediction_layer(x)
  else: #without data augmentation
    x = preprocess_input(inputs)
    x = base_model(x)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(config.dropout)(x)
    outputs = prediction_layer(x)

#------------------FINE TUNING--------------------------------------------------
  fine_tune_at = int(len(base_model.layers) * config.fine_tune_at_fraction) #finding the number of layers to be 'frozen'

  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False  

#------------------OPTIMIZERS---------------------------------------------------
  learning_rate = 1e-3
  decay_rate = learning_rate / config.epochs

  if config.optimizer == 'adam': #Adam optimizer
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate, decay=decay_rate)
  elif config.optimizer == 'nadam': #Nadam optimizer
    optimizer = tf.keras.optimizers.Nadam(lr=learning_rate, decay=decay_rate)
  elif config.optimizer == 'rmsprop': #RMSProp optimizer
    optimizer = tf.keras.optimizers.RMSprop(lr=learning_rate, decay=decay_rate)
  elif config.optimizer == 'nag': #NAG optimizer
    optimizer = tf.keras.optimizers.SGD(lr=learning_rate, decay=decay_rate, nesterov=True)

#-----------------DEFINING THE MODEL--------------------------------------------  
  model = tf.keras.Model(inputs, outputs)

  model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  metrics=['accuracy'])
  
  model.fit(train_dataset, validation_data = validation_dataset, epochs = config.epochs, callbacks = [WandbCallback()])

In [ ]:
import wandb

sweep_config = {
  "name": "My Sweep",
  "method": "random",
  "project": "Assignment02",
  "metric":{
      "name":"val_accuracy",
      "goal":"maximize"
  },
  "parameters": {
        "base": {
            "values": ['IncV3', 'IncResV2', 'Res50', 'Xcep']
        },
        "fine_tune_at_fraction": {
            "values":[1, 0.95, 0.9, 0.85, 0.8]
        }, 
        "epochs": {
            "values":[5,10,15]
        },
        "data_augmentation":{
            "values":['Y',"N"]
        },
        "dropout":{
            "values":[0,0.2,0.3,0.4]
        },  
        "optimizer": {
            "values":['adam', 'nadam','rmsprop','nag']
        }
    }
}

sweep_id = wandb.sweep(sweep_config)

In [ ]:
#wandb.agent(sweep_id, function=swp)